# Análisis de Grandes Volúmenes de Datos
## Proyecto BigData PySpark
### Evidencia 1

Integrantes:

+ Jorge Barón Bracho - A01422588
+ Elda Cristina Morales Sánchez de la Barquera - A00449074
+ Eduardo Selim Martínez Mayorga - A01795167
+ José Arturo Valdivia Rivera - A01795395

In [1]:
#Descargamos las librerias necesarias, iniciamos sesion de Pyspark y cargamos la base de datos
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import countDistinct, count, isnan, when, desc
from pyspark.sql.functions import col, expr, mean, stddev, min, max, length
from pyspark.sql.types import StructType, StructField, StringType, DoubleType
from pyspark.sql.types import NumericType
import math
import pandas as pd

spark = SparkSession.builder.appName("CSVRead").getOrCreate()
df = spark.read.csv("Iowa_Liquor_Sales.csv", header=True, inferSchema=True)

25/05/04 16:43:47 WARN Utils: Your hostname, MacBook-Air-de-Eduardo.local resolves to a loopback address: 127.0.0.1; using 192.168.100.10 instead (on interface en0)
25/05/04 16:43:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/04 16:43:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/05/04 16:44:01 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [2]:
# Cambiamos los nombres de las columnas para facilitar su manejo al programar
df = df.withColumnsRenamed({"Invoice/Item Number": "invoice_number",
"Date": "date",
"Store Number": "store_number",
"Store Name": "store_name",
"Address": "address",
"City": "city",
"Zip Code": "zip_code",
"Store Location": "store_location",
"County Number": "county_number",
"County": "county",
"Category": "category",
"Category Name": "category_name",
"Vendor Number": "vendor_number",
"Vendor Name": "vendor_name",
"Item Number": "item_number",
"Item Description": "item_description",
"Pack": "pack",
"Bottle Volume (ml)": "bottle_volume",
"State Bottle Cost": "bottle_cost",
"State Bottle Retail": "bottle_retail",
"Bottles Sold": "bottles_sold",
"Sale (Dollars)": "sale_dollars",
"Volume Sold (Liters)": "liters_sold",
"Volume Sold (Gallons)": "gallons_sold"})

In [3]:
# Identificamos cuáles son las variables numéricas y cuáles son las variables categóricas
numeric_cols = [col_name for col_name, dtype in df.dtypes if dtype in ('int', 'double')]
textual_cols = [col_name for col_name, dtype in df.dtypes if dtype == 'string']

Es decir, que las variables numéricas son:

In [4]:
print(numeric_cols)

['store_number', 'county_number', 'category', 'vendor_number', 'pack', 'bottle_volume', 'bottle_cost', 'bottle_retail', 'bottles_sold', 'sale_dollars', 'liters_sold', 'gallons_sold']


y las variables categóricas son:

In [5]:
print(textual_cols)

['invoice_number', 'date', 'store_name', 'address', 'city', 'zip_code', 'store_location', 'county', 'category_name', 'vendor_name', 'item_number', 'item_description']


## Análisis de las variables numéricas

In [6]:
numeric_cols = ['pack', 'bottle_volume', 'bottle_cost', 'bottle_retail', 'bottles_sold', 'sale_dollars', 'liters_sold']

In [7]:
#Aqui sacamos las estadisticas para realizar el resumen de las columnas numericas
#Calculamos (mean, std, min, max)
basic_stats_exprs = [
    mean(c).alias(f"{c}_mean") for c in numeric_cols
] + [
    stddev(c).alias(f"{c}_std") for c in numeric_cols
] + [
    min(c).alias(f"{c}_min") for c in numeric_cols
] + [
    max(c).alias(f"{c}_max") for c in numeric_cols
]
basic_stats_df = df.agg(*basic_stats_exprs)
basic_stats_row = basic_stats_df.first().asDict()
#Calculamos los cuartiles (Q1, mediana, Q3)
quartile_exprs = [
    expr(f"percentile_approx({c}, array(0.25, 0.5, 0.75), 10000)").alias(f"{c}_quartiles")
    for c in numeric_cols
]
quartile_df = df.agg(*quartile_exprs)
quartile_row = quartile_df.first().asDict()
#Juntamos todos los datos que calculamos
summary_no = []
for col_name in numeric_cols:
    q1, median, q3 = quartile_row[f"{col_name}_quartiles"]
    summary_no.append({
        "Variable": col_name,
        "Q1": q1,
        "Median": median,
        "Q3": q3,
        "Mean": basic_stats_row[f"{col_name}_mean"],
        "Std": basic_stats_row[f"{col_name}_std"],
        "Min": basic_stats_row[f"{col_name}_min"],
        "Max": basic_stats_row[f"{col_name}_max"],
    })
#Checamos "Summary" por posibles errores y nos hacemos cargo de ellos.
for row in summary_no:
    row["Variable"] = str(row["Variable"])
    for key in row:
        if key != "Variable":
            val = row[key]
            #Comprobamos que el valor no sea "NONE" o "NaN" en caso de que lo sean lo remplazamos por 0.0
            if val is None or (isinstance(val, float) and math.isnan(val)):
                row[key] = 0.0
            else:
                row[key] = float(val)
#Utilizando Pandas sacamos la información del Resumen que realizamos.
pd.set_option('display.max_colwidth', None)
pd.DataFrame(summary_no)

25/05/04 18:59:45 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,Variable,Q1,Median,Q3,Mean,Std,Min,Max
0,pack,6.00,12.00,12.00,12.184921,7.722381,1.0,336.00
1,bottle_volume,750.00,750.00,1000.00,887.205490,634.198109,0.0,378000.00
2,bottle_cost,5.54,8.25,12.59,10.383073,11.810718,0.0,18436.00
3,bottle_retail,8.31,12.39,18.92,15.586002,17.715291,0.0,27654.00
4,bottles_sold,3.00,6.00,12.00,10.694637,29.811181,-360.0,15000.00
5,sale_dollars,33.75,75.00,148.56,142.207275,496.043369,-9720.0,279557.28
6,liters_sold,1.50,4.80,10.50,9.165006,35.625688,-630.0,15000.00


## Análisis de las variables categóricas

In [8]:
textual_cols = ['date', 'store_number', 'county_number', 'category', 'vendor_number', 'address', 'city', 'zip_code',
                'store_location', 'item_number', 'item_description']

In [10]:
#Empezaremos el analisis con las columnas textuales
summary_text = []
for col_name in textual_cols:
    #Sacamos las estadisticas como, valores distintos, espacios vacios, y largo (avg,min,max)
    agg_stats = df.select(
        countDistinct(col(col_name)).alias("distinct_count"),
        count(when((col(col_name).isNull()) | (col(col_name) == ""), col_name)).alias("null_or_empty"),
        mean(length(col(col_name))).alias("avg_len"),
        min(length(col(col_name))).alias("min_len"),
        max(length(col(col_name))).alias("max_len")
    ).collect()[0]
    # Obtenemos los 5 valores mas frecuentes de cada columna
    top_values_df = (
        df.groupBy(col_name)
        .count()
        .orderBy(desc("count"))
        .limit(5)
        .toPandas()
    )
    top_values = top_values_df.to_dict(orient="records")
    #Juntamos todos los resultados en una lista
    summary_text.append({
        "variable": col_name,
        "tipo": "textual",
        "valores_distintos": agg_stats["distinct_count"],
        "valores_nulos_o_vacios": agg_stats["null_or_empty"],
        "longitud_promedio": round(agg_stats["avg_len"], 2) if agg_stats["avg_len"] else None,
        "longitud_minima": agg_stats["min_len"],
        "longitud_maxima": agg_stats["max_len"],
        "top_5_valores_mas_frecuentes": top_values
    })
#Utilizando Pandas sacamos la información del Resumen que realizamos.
pd.set_option('display.max_colwidth', None)
pd.DataFrame(summary_text)

,variable,tipo,valores_distintos,valores_nulos_o_vacios,longitud_promedio,longitud_minima,longitud_maxima,top_5_valores_mas_frecuentes
0,date,textual,2881,0,10.00,10,10,"[{'date': '12/27/2022', 'count': 18965}, {'date': '12/22/2020', 'count': 18002}, {'date': '12/29/2020', 'count': 17722}, {'date': '12/22/2017', 'count': 16674}, {'date': '05/29/2020', 'count': 16650}]"
1,store_number,textual,2903,0,4.00,4,5,"[{'store_number': 2633, 'count': 216425}, {'store_number': 4829, 'count': 183611}, {'store_number': 2190, 'count': 169112}, {'store_number': 2512, 'count': 144290}, {'store_number': 2572, 'count': 137906}]"
2,county_number,textual,99,2028072,1.91,1,2,"[{'county_number': 77.0, 'count': 4459618}, {'county_number': nan, 'count': 2028072}, {'county_number': 57.0, 'count': 2013416}, {'county_number': 82.0, 'count': 1472224}, {'county_number': 7.0, 'count': 1353303}]"
3,category,textual,114,16974,7.00,6,7,"[{'category': 1031100, 'count': 2537263}, {'category': 1012100, 'count': 2501083}, {'category': 1011200, 'count': 1638744}, {'category': 1031200, 'count': 1296294}, {'category': 1031080, 'count': 1265974}]"
4,vendor_number,textual,452,9,2.79,2,3,"[{'vendor_number': 260, 'count': 4325813}, {'vendor_number': 65, 'count': 2414304}, {'vendor_number': 434, 'count': 2291204}, {'vendor_number': 421, 'count': 2143346}, {'vendor_number': 370, 'count': 1536375}]"
5,address,textual,2907,82627,15.88,8,55,"[{'address': '3221 SE 14TH ST', 'count': 293694}, {'address': '1501 MICHIGAN AVE', 'count': 183611}, {'address': '1460 2ND AVE', 'count': 169112}, {'address': '1720 WATERFRONT DR', 'count': 144290}, {'address': '6301 UNIVERSITY', 'count': 137906}]"
6,city,textual,479,82626,9.20,3,17,"[{'city': 'DES MOINES', 'count': 2243599}, {'city': 'CEDAR RAPIDS', 'count': 1692339}, {'city': 'DAVENPORT', 'count': 1117990}, {'city': 'WATERLOO', 'count': 817403}, {'city': 'COUNCIL BLUFFS', 'count': 763224}]"
7,zip_code,textual,513,82693,5.00,5,5,"[{'zip_code': '50010', 'count': 627215}, {'zip_code': '52402', 'count': 595565}, {'zip_code': '52240', 'count': 557509}, {'zip_code': '50613', 'count': 503721}, {'zip_code': '50317', 'count': 464611}]"
8,store_location,textual,5502,2495742,28.70,23,45,"[{'store_location': None, 'count': 2495742}, {'store_location': 'POINT (-93.596754 41.554101)', 'count': 189532}, {'store_location': 'POINT (-93.613739 41.60572)', 'count': 155441}, {'store_location': 'POINT (-93.619787 41.60566)', 'count': 151647}, {'store_location': 'POINT (-92.435236 42.512789)', 'count': 132132}]"
9,item_number,textual,12242,0,4.98,3,7,"[{'item_number': '11788', 'count': 262056}, {'item_number': '36308', 'count': 247224}, {'item_number': '35918', 'count': 166949}, {'item_number': '64858', 'count': 157391}, {'item_number': '36904', 'count': 142933}]"
